Using business.json file and loading it as a dataframe

In [ ]:
import pandas as pd


business_json_path = '/Users/zainabazizzaveri/Downloads/yelp_dataset/business.json'
df_b = pd.read_json(business_json_path, lines=True, orient=str)

Dropping extra information from the dataframe

In [7]:
drop_columns = ['hours','is_open','review_count']
df_b = df_b.drop(drop_columns, axis=1)

Checking size of file to make sure all data was loaded properly

In [8]:
df_b.size

2118699

Splitting multiple values from category column to independent records

In [72]:
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

Finding count of total number of unqiue categories and businesses per category

In [73]:
df_explode.categories.value_counts()

Restaurants         59371
Shopping            31878
Food                29989
Home Services       19729
Beauty & Spas       19370
                    ...  
Hang Gliding            1
Beer Hall               1
Halfway Houses          1
Eastern European        1
Drive-Thru Bars         1
Name: categories, Length: 1300, dtype: int64

From 1300 categories, using ms-excel to find only relevant categories to food and importing those 287 categories from a csv file. Creating a dataframe for 287 categories.

In [61]:
categories_file = '/Users/zainabazizzaveri/Downloads/cat_vals.csv'
df_cat = pd.read_csv(categories_file)
df_cat['Category']

0                Acai Bowls
1               Active Life
2                    Afghan
3                   African
4                Allergists
               ...         
282    Wine Tasting Classes
283       Wine Tasting Room
284              Wine Tours
285                Wineries
286                   Wraps
Name: Category, Length: 287, dtype: object

Limiting the records from the business.json dataframe to include records only for relveant categories

In [ ]:
business = df_explode[df_explode['categories'].isin(df_cat['Category'])]

Dropping some other extra info from business dataframe

In [94]:
business = business.drop(['stars','attributes','categories'],axis=1)

Elimitating duplicate business IDs to get unique businesses

In [95]:
x = business.drop_duplicates('business_id')
x

,business_id,name,address,city,state,postal_code,latitude,longitude
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223
7,gbQN7vr_caG_A1ugSmGhWg,Supercuts,"4545 E Tropicana Rd Ste 8, Tropicana",Las Vegas,NV,89121,36.099872,-115.074574
...,...,...,...,...,...,...,...,...
192598,vIAEWbTJc657yN8I4z7whQ,Starbucks,"8164 S. Las Vegas Blvd., #100",Las Vegas,NV,89123,36.041407,-115.171698
192601,jJoPLIhCjkGXy59I4Jl6zw,JW Marriott Phoenix Desert Ridge Resort & Spa,5350 E Marriott Dr,Phoenix,AZ,85054,33.683910,-111.966235
192602,go-_xdHHSufchOeZ3kkC8w,Cedar Green Wine & Cheese,2179 S Green Rd,University Heights,OH,44121,41.500621,-81.518241
192603,p1eUTUB_5hf1enoKYBV3DA,Ken Stewart's Tre Belle,1911 N Cleveland-Massillon Rd,Bath,OH,44210,41.187547,-81.635791


Importing the reviews.json file using incremental batch processing

In [96]:
review_json_path = '/Users/zainabazizzaveri/Downloads/yelp_dataset/review.json'

In [97]:
size = 100000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int}, chunksize=size)
chunk_list = []
try:
    for chunk_review in review:
        # Drop columns that aren't needed
        chunk_review = chunk_review.drop(['review_id','user_id','stars','useful','funny','cool'], axis=1)
        # Inner merge with edited business file so only reviews related to the business remain
        chunk_merged = pd.merge(x,chunk_review, on='business_id', how='inner')
        # Show feedback on progress
        print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
        chunk_list.append(chunk_merged)
        # After trimming down the review file, concatenate all relevant data back to one dataframe
        
    df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
except ValueError:
    print("data error at ", chunk_review)
print("done") 



85164 out of 100,000 related reviews
85231 out of 100,000 related reviews
85057 out of 100,000 related reviews
85262 out of 100,000 related reviews
85269 out of 100,000 related reviews
84027 out of 100,000 related reviews
83541 out of 100,000 related reviews
84997 out of 100,000 related reviews
85147 out of 100,000 related reviews
85001 out of 100,000 related reviews
85121 out of 100,000 related reviews
85048 out of 100,000 related reviews
82816 out of 100,000 related reviews
83920 out of 100,000 related reviews
84566 out of 100,000 related reviews
84479 out of 100,000 related reviews
84513 out of 100,000 related reviews
84588 out of 100,000 related reviews
84711 out of 100,000 related reviews
82024 out of 100,000 related reviews
85111 out of 100,000 related reviews
85279 out of 100,000 related reviews
85231 out of 100,000 related reviews
85276 out of 100,000 related reviews
85571 out of 100,000 related reviews
84094 out of 100,000 related reviews
82533 out of 100,000 related reviews
8

The dataframe with both reviews and their corresponding businesses

In [98]:
df

,business_id,name,address,city,state,postal_code,latitude,longitude,text,date
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,Don't go here if you expect consistent or reas...,2015-09-24 00:02:23
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,My girlfriend and I went for dinner at Emerald...,2017-01-27 21:54:30
2,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,We've always been there on a Sunday so we were...,2013-06-24 23:11:30
3,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,"***No automatic doors, not baby friendly!*** I...",2016-01-04 12:59:22
4,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,"Horrible service,\nI went there tonight with m...",2014-05-09 02:38:43
...,...,...,...,...,...,...,...,...,...,...
5662668,go-_xdHHSufchOeZ3kkC8w,Cedar Green Wine & Cheese,2179 S Green Rd,University Heights,OH,44121,41.500621,-81.518241,Let me start by saying I've spent hundreds of ...,2011-01-02 21:50:22
5662669,go-_xdHHSufchOeZ3kkC8w,Cedar Green Wine & Cheese,2179 S Green Rd,University Heights,OH,44121,41.500621,-81.518241,Last night I had an awful experience at Cedar ...,2013-04-11 00:30:26
5662670,p1eUTUB_5hf1enoKYBV3DA,Ken Stewart's Tre Belle,1911 N Cleveland-Massillon Rd,Bath,OH,44210,41.187547,-81.635791,I wasnt impressed. The place is beautiful and ...,2012-04-25 16:51:38
5662671,p1eUTUB_5hf1enoKYBV3DA,Ken Stewart's Tre Belle,1911 N Cleveland-Massillon Rd,Bath,OH,44210,41.187547,-81.635791,Last night some friends and I dined at this es...,2018-02-22 22:32:44


Dropping extra info

In [100]:
drop_columns = ['business_id']
df_new = df.drop(drop_columns, axis=1)

Duplicate reviews removal

In [102]:
dup = df_new.drop_duplicates('text')

In [106]:
dup

,name,address,city,state,postal_code,latitude,longitude,text,date
0,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,Don't go here if you expect consistent or reas...,2015-09-24 00:02:23
1,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,My girlfriend and I went for dinner at Emerald...,2017-01-27 21:54:30
2,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,We've always been there on a Sunday so we were...,2013-06-24 23:11:30
3,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,"***No automatic doors, not baby friendly!*** I...",2016-01-04 12:59:22
4,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,"Horrible service,\nI went there tonight with m...",2014-05-09 02:38:43
...,...,...,...,...,...,...,...,...,...
5662668,Cedar Green Wine & Cheese,2179 S Green Rd,University Heights,OH,44121,41.500621,-81.518241,Let me start by saying I've spent hundreds of ...,2011-01-02 21:50:22
5662669,Cedar Green Wine & Cheese,2179 S Green Rd,University Heights,OH,44121,41.500621,-81.518241,Last night I had an awful experience at Cedar ...,2013-04-11 00:30:26
5662670,Ken Stewart's Tre Belle,1911 N Cleveland-Massillon Rd,Bath,OH,44210,41.187547,-81.635791,I wasnt impressed. The place is beautiful and ...,2012-04-25 16:51:38
5662671,Ken Stewart's Tre Belle,1911 N Cleveland-Massillon Rd,Bath,OH,44210,41.187547,-81.635791,Last night some friends and I dined at this es...,2018-02-22 22:32:44


Loading a csv file with all state abbreviations of US states to limit the data only to US region

In [105]:
states_file = '/Users/zainabazizzaveri/Downloads/american_states.csv'
df_st = pd.read_csv(states_file)
df_st['State']

0     AL
1     AK
2     AZ
3     AR
4     CA
5     CO
6     CT
7     DE
8     DC
9     FL
10    GA
11    HI
12    ID
13    IL
14    IN
15    IA
16    KS
17    KY
18    LA
19    ME
20    MD
21    MA
22    MI
23    MN
24    MS
25    MO
26    MT
27    NE
28    NV
29    NH
30    NJ
31    NM
32    NY
33    NC
34    ND
35    OH
36    OK
37    OR
38    PA
39    RI
40    SC
41    SD
42    TN
43    TX
44    UT
45    VT
46    VA
47    WA
48    WV
49    WI
50    WY
Name: State, dtype: object

Removing states which are not in US

In [108]:
final = dup[dup['state'].isin(df_st['State'])]

In [109]:
final

,name,address,city,state,postal_code,latitude,longitude,text,date
0,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,Don't go here if you expect consistent or reas...,2015-09-24 00:02:23
27,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,"Husband was craving Chicken Teriyaki & gyoza, ...",2014-02-24 02:51:56
28,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,Went there Saturday noon they open at 12pm but...,2014-07-13 20:28:18
29,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,Went to this restaurant on Saturday as I had a...,2018-05-09 00:39:36
30,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,This place will always have a place in my stom...,2017-10-25 13:58:04
...,...,...,...,...,...,...,...,...,...
5662668,Cedar Green Wine & Cheese,2179 S Green Rd,University Heights,OH,44121,41.500621,-81.518241,Let me start by saying I've spent hundreds of ...,2011-01-02 21:50:22
5662669,Cedar Green Wine & Cheese,2179 S Green Rd,University Heights,OH,44121,41.500621,-81.518241,Last night I had an awful experience at Cedar ...,2013-04-11 00:30:26
5662670,Ken Stewart's Tre Belle,1911 N Cleveland-Massillon Rd,Bath,OH,44210,41.187547,-81.635791,I wasnt impressed. The place is beautiful and ...,2012-04-25 16:51:38
5662671,Ken Stewart's Tre Belle,1911 N Cleveland-Massillon Rd,Bath,OH,44210,41.187547,-81.635791,Last night some friends and I dined at this es...,2018-02-22 22:32:44


Writing final dataframe with required details to csv file

In [110]:
csv_name = "/Users/zainabazizzaveri/Downloads/all_filtered_reviews.csv"
final.to_csv(csv_name, index=False)